In [138]:
import pandas as pd
import numpy as np


In [220]:

budget_23_xls = pd.ExcelFile('TALOUSARVIO 23.xlsx')
plan_23_xls = pd.ExcelFile('TOIMINTASUUNNITELMA 23.xlsx')

budget_23_sheet_sp3 = budget_23_xls.parse("KESKINEN SP3", header=[9,10,11])
budget_23_sheet_sp4 = budget_23_xls.parse("POHJOINEN SP4", header=[9,10,11])
budget_23_sheet_sp5 = budget_23_xls.parse("KOILLINEN SP5", header=[9,10,11])

budget_23_sheet=pd.concat([budget_23_sheet_sp3,budget_23_sheet_sp4,budget_23_sheet_sp5],axis=0)


# plan_23_sheet_1 = plan_23_xls.parse("Uudisrakentaminen", header=[0,1])

# pohjoinen_sp4 = pd.read_excel(xls,'POHJOINEN SP4')
# koillinen_sp5 = pd.read_excel(xls,'KOILLINEN SP5')


In [221]:
budget_23_sheet

Kohde          Kategoria      Vaikutus  \
                    Unnamed: 0_level_1 Unnamed: 1_level_1 asuntotuotan.   
                    Unnamed: 0_level_2 Unnamed: 1_level_2    Kyllä / Ei   
0                                  NaN                NaN           NaN   
1    8 03 01 01 UUDISRAKENTAMINEN YHT.                NaN           NaN   
2                                  NaN                NaN           NaN   
3         8 03 01 02 PERUSPARANTAMINEN                NaN           NaN   
4                                  NaN                NaN           NaN   
..                                 ...                ...           ...   
387                                NaN                NaN           NaN   
388                                NaN                NaN           NaN   
389                                NaN                NaN           NaN   
390                                NaN                NaN           NaN   
391                                NaN                NaN           NaN   

               Vaikuts Suorite-  Yksikkö- Kustannus-  Työvaihe  Valm. Tot.  \
                  kem.    määrä kustannus    ennuste      2023   2022 2023   
    Unnamed: 3_level_2       m2    € / m2     1000 € (M+K+P+V)      %    %   
0                  NaN      NaN       NaN        NaN       NaN    NaN  NaN   
1                  NaN      NaN       NaN        NaN       NaN    NaN  NaN   
2                  NaN      NaN       NaN        NaN       NaN    NaN  NaN   
3                  NaN      NaN       NaN        NaN       NaN    NaN  NaN   
4                  NaN      NaN       NaN        NaN       NaN    NaN  NaN   
..                 ...      ...       ...        ...       ...    ...  ...   
387                NaN      NaN       NaN        NaN       NaN   K5.1  NaN   
388                NaN      NaN       NaN        NaN       NaN   K5.2  NaN   
389                NaN      NaN       NaN        NaN       NaN   K5.3  NaN   
390                NaN      NaN       NaN        NaN       NaN   K5.4  NaN   
391                NaN      NaN       NaN        NaN       NaN   K5.5  NaN   

     ...    TSE      Alustava investointiohjelma 2026-2032                \
     ...   2025                                       2026   2027   2028   
     ... 1000 €                                     1000 € 1000 € 1000 €   
0    ...    NaN                                        NaN    NaN    NaN   
1    ...    NaN                                        NaN    NaN    NaN   
2    ...    NaN                                        NaN    NaN    NaN   
3    ...    NaN                                        NaN    NaN    NaN   
4    ...    NaN                                        NaN    NaN    NaN   
..   ...    ...                                        ...    ...    ...   
387  ...    NaN                                        NaN    NaN    NaN   
388  ...    NaN                                        NaN    NaN    NaN   
389  ...    NaN                                        NaN    NaN    NaN   
390  ...    NaN                                        NaN    NaN    NaN   
391  ...    NaN                                        NaN    NaN    NaN   

                                               HUOM              
      2029   2030   2031   2032 Unnamed: 29_level_1    HKR / PW  
    1000 € 1000 € 1000 € 1000 € Unnamed: 29_level_2 HANKENUMERO  
0      NaN    NaN    NaN    NaN                 NaN         NaN  
1      NaN    NaN    NaN    NaN                 NaN         NaN  
2      NaN    NaN    NaN    NaN                 NaN         NaN  
3      NaN    NaN    NaN    NaN                 NaN         NaN  
4      NaN    NaN    NaN    NaN                 NaN         NaN  
..     ...    ...    ...    ...                 ...         ...  
387    NaN    NaN    NaN    NaN                 NaN         NaN  
388    NaN    NaN    NaN    NaN                 NaN         NaN  
389    NaN    NaN    NaN    NaN                 NaN         NaN  
390    NaN    NaN    NaN    NaN 

In [222]:
# Dropping columns we don't need
budget_23_sheet = budget_23_sheet.drop(
    budget_23_sheet.columns[[3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]],
    axis=1,
)

# CHANGE THIS HERE TO ONLY GET PROJECTS AND NOT MISS ANY PROJECT
budget_23_sheet = budget_23_sheet[
    budget_23_sheet[budget_23_sheet.columns[15]].notna()
]
budget_23_sheet = budget_23_sheet.replace({np.nan: None, "?": None})

# plan_23_sheet_1 = plan_23_sheet_1.drop(
#     plan_23_sheet_1.columns[[5, 6, 8,23,24,25]],
#     axis=1,
# )
# plan_23_sheet_1 = plan_23_sheet_1[
#     plan_23_sheet_1[plan_23_sheet_1.columns[21]].notna()
# ]
# plan_23_sheet_1 = plan_23_sheet_1.replace({np.nan: None, "?": None})




In [223]:
budget_23_sheet

Kohde          Kategoria  \
                                    Unnamed: 0_level_1 Unnamed: 1_level_1   
                                    Unnamed: 0_level_2 Unnamed: 1_level_2   
16       Siltasaarenkatu (Toinen linja - Kolmas linja)                 K1   
19   Hakaniementori + (Länsipää Miina Sillanpään katu)               K5.1   
20             Siltasaarenkatu (Hakaniemenr - Hämeent)               K5.1   
21          Signe Branderin terassi (itä+länsi) + kuja               K5.1   
22                                  Näkinkulku (pinta)               K5.1   
..                                                 ...                ...   
234                                    Vaskipellonkuja                 K4   
236                                       Vankkurikuja                 K4   
237                                           Iespolku                 K4   
240                                   Vaskipellonpolku                 K4   
242                                        Kesantokuja                 K4   

         Vaikutus Kustannus-    TAE    TSE    TSE  \
    asuntotuotan.    ennuste   2023   2024   2025   
       Kyllä / Ei     1000 € 1000 € 1000 € 1000 €   
16              K     1000.0   None   None   None   
19              E     3000.0   None   None   None   
20              K      500.0   None   None   None   
21              K     3500.0   None   None   None   
22              K      300.0   None   None   None   
..            ...        ...    ...    ...    ...   
234          None       None   None   None   None   
236          None       None   None   None   None   
237          None       None   None   None   None   
240          None       None   None   None   None   
242          None       None   None   None   None   

         Alustava investointiohjelma 2026-2032                              \
                                          2026   2027   2028   2029   2030   
                                        1000 € 1000 € 1000 € 1000 € 1000 €   
16                                        None   None   None   None   None   
19                                        None   None   None   None   None   
20                                        None   None   None   None   None   
21                                        None   None   None   None   None   
22                                        None   None   None   None   None   
..                                         ...    ...    ...    ...    ...   
234                                       None   None   None   None   None   
236                                       None   None   None   None   None   
237                                       None   None   None   None   None   
240                                       None   None   None   None   None   
242                                       None   None   None   None   None   

                                                                HUOM  \
      2031   2032                                Unnamed: 29_level_1   
    1000 € 1000 €                                Unnamed: 29_level_2   
16    None   None  . 23 0,3 milj. Kivityös 24 0,5 milj.  TARKISTA...   
19    None   None    Suunnittelun aloitus vasta 25 ja toteutus 26+27   
20    None   None                                               None   
21    None   None                                               None   
22    None   None                                               None   
..     ...    ...                                                ...   
234   None   None                                               None   
236   None   None                                               None   
237   None   None                                               None   
240   None   None                                               None   
242   None   None                                               None   

                 
       HKR / PW  
    HANKENUMERO  
16         2637  
19          191  
20          839  
21         3174  

In [224]:
budget_23_sp3_data = [
    {
        "name": name,
        "category": category,
        "effectHousing": effectHousing,
        "costForecast": costForecast,
        "budgetProposalCurrentYearPlus1": bpCPlus1,
        "budgetProposalCurrentYearPlus2": bpCPlus2,
        "preliminaryCurrentYearPlus3": pcyPlus3,
        "preliminaryCurrentYearPlus4": pcyPlus4,
        "preliminaryCurrentYearPlus5": pcyPlus5,
        "preliminaryCurrentYearPlus6": pcyPlus6,
        "preliminaryCurrentYearPlus7": pcyPlus7,
        "preliminaryCurrentYearPlus8": pcyPlus8,
        "preliminaryCurrentYearPlus9": pcyPlus9,
        "preliminaryCurrentYearPlus10": pcyPlus10,
        "note_content": note_content,
        "hkrId": hkrId,
    }
    for name, category, effectHousing, costForecast, bpCPlus1, bpCPlus2, pcyPlus3, pcyPlus4, pcyPlus5, pcyPlus6, pcyPlus7, pcyPlus8, pcyPlus9, pcyPlus10, note_content, hkrId in zip(
        budget_23_sheet.iloc[:, 0],
        budget_23_sheet.iloc[:, 1],
        budget_23_sheet.iloc[:, 2],
        budget_23_sheet.iloc[:, 3],
        budget_23_sheet.iloc[:, 4],
        budget_23_sheet.iloc[:, 5],
        budget_23_sheet.iloc[:, 6],
        budget_23_sheet.iloc[:, 7],
        budget_23_sheet.iloc[:, 8],
        budget_23_sheet.iloc[:, 9],
        budget_23_sheet.iloc[:, 10],
        budget_23_sheet.iloc[:, 11],
        budget_23_sheet.iloc[:, 12],
        budget_23_sheet.iloc[:, 13],
        budget_23_sheet.iloc[:, 14],
        budget_23_sheet.iloc[:, 15],
        
    )
]

In [225]:
budget_23_sp3_data

[{'name': 'Siltasaarenkatu (Toinen linja - Kolmas linja)',
  'category': 'K1',
  'effectHousing': 'K',
  'costForecast': 1000.0,
  'budgetProposalCurrentYearPlus1': None,
  'budgetProposalCurrentYearPlus2': None,
  'preliminaryCurrentYearPlus3': None,
  'preliminaryCurrentYearPlus4': None,
  'preliminaryCurrentYearPlus5': None,
  'preliminaryCurrentYearPlus6': None,
  'preliminaryCurrentYearPlus7': None,
  'preliminaryCurrentYearPlus8': None,
  'preliminaryCurrentYearPlus9': None,
  'preliminaryCurrentYearPlus10': None,
  'note_content': '. 23 0,3 milj. Kivityös 24 0,5 milj.  TARKISTA VARAUS 24?',
  'hkrId': 2637},
 {'name': 'Hakaniementori + (Länsipää Miina Sillanpään katu)',
  'category': 'K5.1',
  'effectHousing': 'E',
  'costForecast': 3000.0,
  'budgetProposalCurrentYearPlus1': None,
  'budgetProposalCurrentYearPlus2': None,
  'preliminaryCurrentYearPlus3': None,
  'preliminaryCurrentYearPlus4': None,
  'preliminaryCurrentYearPlus5': None,
  'preliminaryCurrentYearPlus6': None,
  